In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = pd.read_csv('songdata.csv' ,encoding='latin-1')

In [ ]:
pip install spotipy

     |████████████████████████████████| 153kB 6.8MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

cid = '7f2ab65404b64bbc99e22b2efcb17983'
secret = 'aa855c61f03f4077ac50a78fa35c71c6'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  duration_ms = meta['duration_ms']
  popularity = meta['popularity']

  #year
  try:
    year = int(release_date[:4])
    if(year<1900 and year>2021):
      year = 0
  except:
    year = 0

  # features
  if 'acousticness' in features[0]:
    acousticness = features[0]['acousticness']
  else:
    acousticness = 0
  danceability = features[0]['danceability'] if features[0]['danceability']!=None else 0
  energy = features[0]['energy'] if features[0]['energy']!=None else 0
  instrumentalness = features[0]['instrumentalness'] if features[0]['instrumentalness']!=None else 0
  liveness = features[0]['liveness'] if features[0]['liveness']!=None else 0
  loudness = features[0]['loudness'] if features[0]['loudness']!=None else 0
  speechiness = features[0]['speechiness'] if features[0]['speechiness']!=None else 0
  tempo = features[0]['tempo'] if features[0]['tempo']!=None else 0


  track = [name, album, artist, release_date]
  track2 = [duration_ms, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, year]
  return track, track2

In [ ]:
new_df = pd.DataFrame(columns=['id', 'name', 'artist', 'album', 'release_date', 'duration_ms', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'year'])
cat = [0, 1, 3, 7, 8, 10, 11, 12, 13, 14, 16, 18, 19]
categories = sp.categories(country="IN")['categories']['items']
time.sleep(0.5)
# for selected categories
for i in cat:
  id = categories[i]['id']
  print("category: ", id)
  # find all playlists
  playlists = sp.category_playlists(id, country="IN")['playlists']['items']
  time.sleep(0.5)
  for j in range(len(playlists)):
    pid = playlists[j]['id']
    # find top 50 tracks
    tracks = sp.playlist_tracks(pid, limit=50)['items']
    time.sleep(0.5)
    try:
      for k in range(len(tracks)):
        songs = []
        tid = tracks[k]['track']['id']
        track1, track2 = getTrackFeatures(tid)
        track1.insert(0,tid)
        track1.extend(track2)
        songs.append(track1)
        record = pd.DataFrame(songs, columns = ['id', 'name', 'album', 'artist', 'release_date', 'duration_ms', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'year'])
        new_df = pd.concat([new_df, record], ignore_index = True)
    except:
      pass

new_df.head()

category:  toplists
category:  at_home
category:  bollywood
category:  indie_alt
category:  pop
category:  marathi
category:  devotional
category:  indian_classical
category:  romance
category:  kpop
category:  party
category:  edm_dance
category:  decades


,id,name,artist,album,release_date,duration_ms,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,year
0,748mdHapucXQri7IAO8yFK,Kiss Me More (feat. SZA),Kiss Me More (feat. SZA),Doja Cat,2021-04-09,208866,96,0.762,0.23500,0.762,0.701,0.000158,0.1230,-3.541,0.0286,110.968,2021
1,463CkQjx2Zk1yXoBuierM9,Levitating (feat. DaBaby),Levitating (feat. DaBaby),Dua Lipa,2020-10-01,203064,91,0.702,0.00883,0.702,0.825,0.000000,0.0674,-3.787,0.0601,102.977,2020
2,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),Justice,Justin Bieber,2021-03-19,198081,100,0.677,0.32100,0.677,0.696,0.000000,0.4200,-6.181,0.1190,90.030,2021
3,37BZB0z9T8Xu7U3e65qxFy,Save Your Tears (with Ariana Grande) (Remix),Save Your Tears (Remix),The Weeknd,2021-04-23,191013,92,0.650,0.02150,0.650,0.825,0.000024,0.0936,-4.645,0.0325,118.091,2021
4,3BZEcbdtXQSo7OrvKRJ6mb,MONTERO (Call Me By Your Name),MONTERO (Call Me By Your Name),Lil Nas X,2021-03-26,137875,88,0.610,0.29700,0.610,0.508,0.000000,0.3840,-6.682,0.1520,178.818,2021


In [ ]:
new_df = pd.read_csv('sdata.csv', encoding='latin-1')

### Content Based Recommender

1. User enters the song name
2. Extract the track ID from the name followed by the features
3. Cosine similarity to find the TOP 5 similar songs.
4. Display all the recommended songs with details (image, title, artist, genre)

In [ ]:
def TFIDF(data, col):
  #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
  tfidf = TfidfVectorizer(stop_words='english')
  #Construct the required TF-IDF matrix by fitting and transforming the data
  tfidf1 = tfidf.fit_transform(data[col])
  return tfidf1

In [ ]:
def get_recommendation_diff(new_df, record):
  temp_df = new_df[['id','name', 'album', 'artist', 'release_date']]
  temp_df = pd.concat([temp_df, record], ignore_index = True)

  col = ['name', 'album', 'artist', 'release_date']
  data = pd.DataFrame(columns=col)
  id = []
  for i in col:
    tf = TFIDF(temp_df, i)
    cosine_sim = linear_kernel(tf, tf)
    data[i] = cosine_sim[-1]
    d1 = data.sort_values(by=[i], ascending=False)
    id.append(list(d1.head(7).index))

  tid = []
  for i in range(4):
    track_id = []
    for j in id[i]:
      track_id.append(temp_df.iloc[j, 0])
    tid.append(track_id)
  return tid

In [ ]:
song = input('Enter a song name:')
track_id = sp.search(q='track:' + song, type='track')
id = track_id['tracks']['items'][0]['id']
song_name = sp.track(id)['name']

tracks = []
track, _ = getTrackFeatures(id)
track.insert(0, id)
tracks.append(track)

# create record
record = pd.DataFrame(tracks, columns = ['id', 'name', 'album', 'artist', 'release_date'])
record.head()

tid = get_recommendation_diff(new_df, record)

prev = song_name
col = ['name', 'album', 'artist', 'release_date']
for i in range(4):
  print("Recommendations based on", col[i])
  for j in tid[i]:
      track = sp.track(j)
      time.sleep(0.5)
      if(track['name']==prev):
       continue
      prev = track['name']
      print('track    : ' + track['name'])
      print('audio    : ' + str(track['preview_url']))
      print('cover art: ' + str(track['album']['images'][0]['url']))
      print()

Enter a song name:Kal Ho Naa Ho
Recommendations based on name
track    : Tumhen Ho Na Ho
audio    : https://p.scdn.co/mp3-preview/2f7e74bd4c218d1a8485a7b82cbbb7c177fd6138?cid=7f2ab65404b64bbc99e22b2efcb17983
cover art: https://i.scdn.co/image/ab67616d0000b273ce40ee9cc7c0e64f472297a3

track    : Tum Ho
audio    : https://p.scdn.co/mp3-preview/ae106c3d2357b9fbf009be8240cca3321a96e122?cid=7f2ab65404b64bbc99e22b2efcb17983
cover art: https://i.scdn.co/image/ab67616d0000b27354e544672baa16145d67612b

track    : Ho Hey
audio    : https://p.scdn.co/mp3-preview/57e57e6eb594e55de01b85cea4d470243f6121f5?cid=7f2ab65404b64bbc99e22b2efcb17983
cover art: https://i.scdn.co/image/ab67616d0000b2734241433fb84b10663492c0e3

track    : Aur Ho
audio    : https://p.scdn.co/mp3-preview/308048ebc3636fdb34d8ce105378f0dea936379e?cid=7f2ab65404b64bbc99e22b2efcb17983
cover art: https://i.scdn.co/image/ab67616d0000b27354e544672baa16145d67612b

track    : Saath Ho
audio    : https://p.scdn.co/mp3-preview/54bc6f4e3495

### Collaborative Filtering Based Recommender (KNN)

1.   Input song name and extracr ID
2.   Extract Features and make a dataframe (numeric columns)
3.   K nearest neighbors


# New Section

In [ ]:
def euclid(a, b):
  distance = np.linalg.norm(a - b)
  return distance

def getNeighbors(record, data, K):
    distances = []

    for i in range(data.shape[0]):
      if data.iloc[i,0] != record.iloc[0,0]:
        dist = euclid(record.iloc[0, 1:].values, data.iloc[i, 1:].values)
        distances.append((data.iloc[i,0], dist))

    distances.sort(key=lambda x : x[1])
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

In [ ]:
track= input('Enter a Song Name: ')
track_id = sp.search(q='track:' + track, type='track')
id = track_id['tracks']['items'][0]['id']
track_name = sp.track(id)['name']

tracks = []
_, track = getTrackFeatures(id)
track.insert(0,id)
tracks.append(track)

# create dataset
record = pd.DataFrame(tracks, columns = ['id', 'duration_ms', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'year'])
data = new_df[['id', 'duration_ms', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'year']]

K = 10

neighbors = getNeighbors(record, data, K)

prev = [track_name]
print('\nRecommended songs: \n')
for neighbor in neighbors:
  # Getting images,audio from Track ID
    track = sp.track(neighbor)
    if(track['name'] in prev):
      continue
    prev.append(track['name'])
    print('track    : ' + track['name'])
    print('audio    : ' + str(track['preview_url']))
    print('cover art: ' + str(track['album']['images'][0]['url']))
    print()

Enter a Song Name: New Rules

Recommended songs: 

track    : All Your Exes
audio    : None
cover art: https://i.scdn.co/image/ab67616d0000b273e17a1f8123db65688773cb27

track    : SLOW DANCING IN THE DARK
audio    : https://p.scdn.co/mp3-preview/483355f39bb264b9828633561ab14a7a48e75270?cid=7f2ab65404b64bbc99e22b2efcb17983
cover art: https://i.scdn.co/image/ab67616d0000b27360ba1d6104d0475c7555a6b2

track    : burn
audio    : https://p.scdn.co/mp3-preview/00a98b949f1de65e34384226a9e4091c025bdfa7?cid=7f2ab65404b64bbc99e22b2efcb17983
cover art: https://i.scdn.co/image/ab67616d0000b273d90cef642edf69504f7155c8

track    : DDU-DU DDU-DU
audio    : None
cover art: https://i.scdn.co/image/ab67616d0000b27398135e0d2d1675f2b502f01c

track    : Oh Sanam
audio    : https://p.scdn.co/mp3-preview/3056dc54b18568a8a85dd0ea01443dabb233403a?cid=7f2ab65404b64bbc99e22b2efcb17983
cover art: https://i.scdn.co/image/ab67616d0000b273e1306e7b70b5074f52abc232

track    : 1 Billion Views
audio    : https://p.scdn.